<a href="https://colab.research.google.com/github/MarceloClaro/MarceloClaro-COMPARA-O_DE_AUT-MATOS_CELULARES_CL-SSICOS_E_QUANTICOS/blob/main/COMPARA%C3%87%C3%83O_DE_AUT%C3%94MATOS_CELULARES_CL%C3%81SSICOS_E_QUANTICOS_Performance%2C_robustez_ao_ru%C3%ADdo_e_aplica%C3%A7%C3%B5es_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

# Executar o comando 'nvidia-smi' para obter informações sobre a GPU
gpu_info = subprocess.run(['nvidia-smi'], capture_output=True, text=True).stdout

# Verificar se há falha na conexão com a GPU
if 'failed' in gpu_info.lower():
    print('Não está conectado a uma GPU')
else:
    print(gpu_info)


In [ ]:
from psutil import virtual_memory

# Obter a quantidade total de RAM disponível
ram_gb = virtual_memory().total / 1e9

# Exibir a quantidade de RAM disponível
print(f'Seu tempo de execução tem {ram_gb:.1f} gigabytes de RAM disponível\n')

# Verificar se está usando um tempo de execução com alta RAM
print('Você está usando um tempo de execução com alta RAM!' if ram_gb >= 20 else 'Não está usando um tempo de execução com alta RAM')


In [ ]:
# Instalar pacotes necessários do Qiskit e Seaborn.
!pip install qiskit>=1.0  # Instala a versão necessária do Qiskit para realizar operações de computação quântica.
!pip install qiskit-ibm-runtime  # Instala a biblioteca qiskit-ibm-runtime para execução de circuitos no IBM Quantum Experience.
!pip install 'qiskit[visualization]'  # Instala o pacote de visualização do Qiskit para visualizações gráficas.
!pip install qiskit_aer  # Instala o simulador Aer do Qiskit para simulações quânticas.
!pip install seaborn  # Instala a biblioteca Seaborn para visualização de dados, especialmente gráficos estatísticos.

# Importando bibliotecas necessárias
import numpy as np  # Importa a biblioteca NumPy, que oferece suporte para arrays e operações numéricas de alto desempenho.
import matplotlib.pyplot as plt  # Importa a biblioteca Matplotlib, que permite a criação de gráficos e visualizações de dados.
import seaborn as sns  # Importa a biblioteca Seaborn, que é baseada no Matplotlib e oferece visualizações estatísticas aprimoradas.
from qiskit import QuantumCircuit, transpile  # Importa classes do Qiskit para criação de circuitos quânticos e transpile para otimização de circuitos.
from qiskit.visualization import plot_histogram, plot_bloch_multivector, circuit_drawer  # Importa funções do Qiskit para visualização de histogramas, vetores de Bloch e circuitos.
from qiskit_aer import AerSimulator  # Importa o simulador Aer do Qiskit, que simula a execução de circuitos quânticos.
import pandas as pd  # Importa a biblioteca pandas, que fornece estruturas de dados de alto desempenho e ferramentas de análise de dados.
from scipy.stats import entropy  # Importa a função de entropia da biblioteca scipy, usada para cálculos estatísticos de entropia.
import time  # Importa a biblioteca time, que fornece funções para medir e manipular o tempo.

# Função para aplicar a regra clássica
def apply_classical_rule(rule_number, ca):
    """
    Aplica uma regra de autômato celular clássico a um estado.

    Parameters:
        rule_number (int): Número da regra a ser aplicada.
        ca (np.array): Array representando o estado atual do autômato.

    Returns:
        np.array: Novo estado do autômato após aplicação da regra.
    """
    # Convertemos o número da regra para uma sequência de 0s e 1s (binário).
    # Vamos imaginar que rule_number é 30. Em binário, 30 é 00011110.
    # Isso significa que a regra 30 tem os seguintes resultados:
    # 111 -> 0, 110 -> 0, 101 -> 0, 100 -> 1, 011 -> 1, 010 -> 1, 001 -> 1, 000 -> 0.
    rule_bin = np.array([int(x) for x in np.binary_repr(rule_number, width=8)], dtype=np.int8)

    # Criamos um novo array (lista) para armazenar o próximo estado das células.
    # No começo, todas as células estão "mortas" (0).
    new_ca = np.zeros_like(ca)

    # Para cada célula, exceto a primeira e a última, aplicamos a regra.
    for i in range(1, len(ca) - 1):
        # Pegamos o estado da célula atual (ca[i]) e das suas vizinhas (ca[i-1] e ca[i+1]).
        # Transformamos esses três estados em um número de 0 a 7.
        # Por exemplo, se ca[i-1] = 1, ca[i] = 0, ca[i+1] = 1, o número será 101 em binário, que é 5 em decimal.
        pattern = ca[i - 1] * 4 + ca[i] * 2 + ca[i + 1]
        # Usamos esse número para encontrar o novo estado da célula na regra binária.
        # Se o número é 5, olhamos para a sexta posição na regra binária (00011110) e vemos que o novo estado é 1.
        new_ca[i] = rule_bin[7 - pattern]

    # Retornamos o novo estado das células.
    return new_ca
    """
    Explicações Adicionais
    Autômato Celular:
    Um autômato celular é um modelo matemático que consiste em uma grade de células. Cada célula pode estar em um de vários estados (por exemplo, 0 ou 1).
    As células evoluem em etapas discretas de tempo de acordo com um conjunto de regras baseadas nos estados das células vizinhas.

    Binário:
    O sistema binário é um sistema numérico que usa apenas dois dígitos: 0 e 1. Por exemplo, o número 30 em binário é 11110.

    Regra 30:
    A regra 30 é uma das regras possíveis para um autômato celular unidimensional, onde cada célula e suas duas vizinhas determinam o próximo estado da célula.
    Por exemplo, para a regra 30, se a célula e suas duas vizinhas estão nos estados 0, 1 e 0, respectivamente (ou seja, 010), o próximo estado da célula será 1.
    """

def simulate_classical(rule_number, steps=100, size=101):
    """
    Simula um autômato celular clássico por um número de passos.

    Parameters:
        rule_number (int): Número da regra a ser aplicada.
        steps (int): Número de passos para simular.
        size (int): Tamanho do autômato.

    Returns:
        np.array: Histórico dos estados do autômato durante a simulação.
    """
    # Inicializa o estado do autômato com todas as células em 0, exceto a do meio.
    # Vamos começar com uma linha de células (como quadradinhos) onde todas estão apagadas (0),
    # exceto a célula bem no meio, que está acesa (1).
    state = np.zeros(size, dtype=np.int8)
    state[size // 2] = 1

    # Cria um histórico para armazenar o estado em cada passo.
    # Aqui, criamos um "álbum de fotos" para guardar a imagem da linha de células em cada passo do tempo.
    history = np.zeros((steps, size), dtype=np.int8)
    history[0] = state

    # Aplica a regra clássica para cada passo e atualiza o histórico.
    # Vamos fazer isso várias vezes (o número de passos) para ver como as células mudam ao longo do tempo.
    for t in range(1, steps):
        # Aplicamos a regra ao estado atual para encontrar o novo estado.
        # Isso é como seguir uma receita para saber quais células acender e quais apagar.
        state = apply_classical_rule(rule_number, state)
        # Guardamos o novo estado no álbum.
        # Tiramos uma "foto" da linha de células e guardamos no álbum.
        history[t] = state

    # No final, retornamos o álbum completo com todas as imagens dos estados.
    # Agora, temos uma sequência de imagens que mostra como a linha de células mudou ao longo do tempo.
    return history
    """
    Explicações Adicionais
    Autômato Celular:
    Um autômato celular é um modelo matemático que consiste em uma grade de células.
    Cada célula pode estar em um de vários estados (por exemplo, 0 ou 1).
    As células evoluem em etapas discretas de tempo de acordo com um conjunto de regras baseadas nos estados das células vizinhas.

    Estado Inicial:
    No início, temos uma linha de células onde todas estão apagadas (0), exceto uma célula no meio que está acesa (1).

    Histórico:
    Durante a simulação, mantemos um registro (histórico) dos estados das células em cada passo do tempo.
    É como tirar uma foto da linha de células em cada passo e guardar todas essas fotos em um álbum.

    Aplicação da Regra:
    Em cada passo do tempo, aplicamos a regra específica (definida por rule_number) para determinar o novo estado das células.
    Esta regra nos diz como cada célula deve mudar com base no seu estado atual e no estado das suas vizinhas.
    """

def add_noise_to_classical(history, noise_level=0.01):
    """
    Adiciona ruído a uma simulação de autômato celular clássico.

    Parameters:
        history (np.array): Histórico dos estados do autômato.
        noise_level (float): Nível de ruído a ser adicionado.

    Returns:
        np.array: Histórico com ruído adicionado.
    """
    # Cria uma cópia do histórico original para adicionar ruído.
    # Isso é como fazer uma cópia do seu desenho antes de adicionar manchas de tinta.
    noisy_history = history.copy()

    # Gera ruído aleatório baseado no nível de ruído especificado.
    # Aqui, estamos gerando uma matriz do mesmo tamanho do histórico com valores aleatórios de 0 ou 1.
    # A chance de ser 1 é definida pelo nível de ruído.
    noise = np.random.choice([0, 1], size=history.shape, p=[1-noise_level, noise_level])

    # Aplica o ruído ao histórico usando a operação XOR.
    # XOR é uma operação que compara dois valores. Se forem iguais, resulta em 0; se forem diferentes, resulta em 1.
    # Isso significa que onde temos um valor 1 no ruído, o valor correspondente no histórico original é invertido (0 vira 1 e 1 vira 0).
    noisy_history = np.bitwise_xor(noisy_history, noise)

    # Retorna o histórico com o ruído adicionado.
    return noisy_history

def add_noise_to_quantum_results(quantum_results, noise_level=0.01):
    """
    Adiciona ruído a uma simulação quântica.

    Parameters:
        quantum_results (list): Resultados da simulação quântica.
        noise_level (float): Nível de ruído a ser adicionado.

    Returns:
        list: Resultados com ruído adicionado.
    """
    # Cria uma lista para armazenar os resultados com ruído.
    # Esta lista será usada para guardar os novos resultados, como uma nova folha de papel para desenhar.
    noisy_results = []

    # Itera sobre cada passo da simulação quântica.
    # Estamos passando por cada imagem no álbum (cada momento no tempo).
    for step in quantum_results:
        noisy_step = []

        # Itera sobre cada bloco de resultados.
        # Dentro de cada imagem, estamos olhando para cada parte dela.
        for block in step:
            noisy_block = {}

            # Aplica ruído a cada estado baseado na contagem original.
            # Para cada parte da imagem, estamos adicionando ruído.
            for state, count in block.items():
                # Calcula a nova contagem com ruído.
                # Estamos ajustando o valor original com base no nível de ruído.
                noisy_count = int(count * (1 - noise_level) + np.random.poisson(noise_level * count))

                # Guarda o novo valor no bloco.
                noisy_block[state] = noisy_count

            # Adiciona o bloco com ruído ao passo.
            noisy_step.append(noisy_block)

        # Adiciona o passo com ruído aos resultados.
        noisy_results.append(noisy_step)

    # Retorna os resultados com o ruído adicionado.
    return noisy_results
"""
    Explicações Detalhadas:

    Função add_noise_to_classical:

    Objetivo: Adicionar ruído a uma simulação clássica de autômato celular.

    Passo a Passo:
    Cópia do Histórico: Fazemos uma cópia do estado original do autômato para poder modificar sem alterar o original.
    Geração de Ruído: Criamos uma matriz do mesmo tamanho que o histórico, preenchida com valores aleatórios de 0 ou 1.
    A chance de ser 1 é definida pelo nível de ruído (noise_level).
    Aplicação do Ruído (XOR): Utilizamos a operação XOR para inverter os valores no histórico onde o ruído for 1.
    Isso altera aleatoriamente algumas células do autômato, simulando a presença de ruído.
    Retorno: Retornamos o histórico com o ruído adicionado.

    Função add_noise_to_quantum_results:

    Objetivo: Adicionar ruído aos resultados de uma simulação quântica.

    Passo a Passo:

    Lista para Resultados com Ruído: Criamos uma nova lista para armazenar os resultados modificados.
    Iteração sobre os Passos: Passamos por cada momento no tempo da simulação.
    Iteração sobre os Blocos: Dentro de cada momento, analisamos cada parte da simulação (blocos).
    Aplicação do Ruído: Para cada estado dentro de um bloco, ajustamos a contagem original com base no nível de ruído.
    Utilizamos uma distribuição de Poisson para gerar um valor de ruído e aplicamos isso à contagem original.
    Armazenamento: Guardamos os novos valores no bloco e, em seguida, adicionamos o bloco à lista de passos.
    Retorno: Retornamos os resultados com o ruído adicionado.

    Termos Matemáticos:

    Matriz: Uma tabela de números organizada em linhas e colunas.
    Nível de Ruído (noise_level): A probabilidade de que um valor seja alterado.
    Operação XOR: Uma operação lógica que compara dois bits e retorna 1 se eles forem diferentes, e 0 se forem iguais.
    Distribuição de Poisson: Uma distribuição de probabilidade que descreve a frequência com que eventos ocorrem em um intervalo fixo de tempo ou espaço.
    """

def apply_quantum_rule_based_on_classical_state(qc, classical_state):
    """
    Aplica uma regra quântica baseada no estado clássico inicial.

    Parameters:
        qc (QuantumCircuit): Circuito quântico a ser configurado.
        classical_state (np.array): Estado clássico inicial.

    Returns:
        QuantumCircuit: Circuito quântico configurado.
    """
    # 'n' é o número de qubits (bits quânticos) que temos, que é o mesmo que o tamanho do estado clássico.
    n = len(classical_state)

    # Aplica a porta X (também conhecida como NOT) aos qubits onde o estado clássico é 1.
    # Isso é como inverter o valor dos bits. Se for 1, vira 0; se for 0, vira 1.
    for i in range(n):
        if classical_state[i] == 1:
            qc.x(i)  # Aplica a porta X no qubit 'i'

    # Retorna o circuito quântico configurado.
    return qc
"""
Explicações Detalhadas:

1. **Objetivo da Função:**
   - Esta função pega um circuito quântico (um conjunto de operações que podemos fazer com qubits) e aplica regras a ele com base em um estado clássico (uma lista de valores que são 0 ou 1).
   - O estado clássico é como um desenho de pontos pretos e brancos, onde 0 é branco e 1 é preto. Queremos transformar esses pontos em operações quânticas.

2. **Passo a Passo:**

   - **Parâmetros:**
     - `qc (QuantumCircuit)`: Este é o circuito quântico que queremos configurar. Pense nisso como uma sequência de comandos que nosso computador quântico vai seguir.
     - `classical_state (np.array)`: Esta é a lista de valores 0 e 1 que representa o estado clássico. Imagine isso como um padrão de luzes onde algumas estão acesas (1) e outras estão apagadas (0).

   - **Número de Qubits (`n`):**
     - `n = len(classical_state)`: Calculamos o número de qubits contando quantos elementos temos no estado clássico. Se tivermos 5 elementos, `n` será 5.
     - **Qubits:** São as unidades básicas de informação em computação quântica, semelhantes aos bits na computação clássica.

   - **Aplicação da Porta X (NOT):**
     - `for i in range(n)`: Vamos passar por cada qubit (de 0 até n-1).
     - `if classical_state[i] == 1`: Verificamos se a luz está acesa (se o valor é 1).
     - `qc.x(i)`: Se a luz está acesa, aplicamos a porta X ao qubit correspondente. A porta X é como um interruptor que inverte o estado do qubit:
       - Se o qubit está em 0, ele vira 1.
       - Se o qubit está em 1, ele vira 0.
     - **Matematicamente:** A operação da porta X é representada pela matriz
     \[
     \text{X} = \begin{pmatrix}
     0 & 1 \\
     1 & 0
     \end{pmatrix}
     \].

     Ela troca os estados \(|0\rangle\) e \(|1\rangle\).

   - **Retorno do Circuito:**
     - `return qc`: Depois de configurar o circuito quântico com base no estado clássico, devolvemos esse circuito para ser usado em simulações ou outras operações.

Termos Matemáticos:

- **Porta X (NOT):** Uma operação que inverte o estado de um qubit. É como um interruptor que muda de 0 para 1 ou de 1 para 0.
- **Qubits:** Unidades básicas de informação em um computador quântico, que podem representar 0, 1 ou ambos ao mesmo tempo devido à superposição.
- **Matriz:** Uma tabela de números organizada em linhas e colunas, usada para representar operações matemáticas.

"""

def run_quantum_simulation_from_classical(df_classical, block_size, rule_number):
    """
    Executa simulações quânticas baseadas nos estados clássicos.

    Parameters:
        df_classical (pd.DataFrame): DataFrame com estados clássicos.
        block_size (int): Tamanho do bloco de células.
        rule_number (int): Número da regra clássica aplicada.

    Returns:
        list: Resultados da simulação quântica.
    """
    # Cria um simulador quântico
    simulator = AerSimulator()
    # Lista para armazenar os resultados
    results = []
    # Calcula o número de blocos necessários
    num_blocks = (len(df_classical.columns) + block_size - 1) // block_size
    # Contador para numerar os circuitos
    circuit_count = 1

    # Itera sobre cada linha do DataFrame clássico
    for index, row in df_classical.iterrows():
        # Lista para armazenar os resultados de cada bloco
        block_results = []
        # Itera sobre cada bloco de células
        for block in range(num_blocks):
            # Início do bloco
            start = block * block_size
            # Fim do bloco
            end = min(start + block_size, len(df_classical.columns))
            # Estado do bloco
            block_state = row.values[start:end]
            # Cria um circuito quântico para o bloco
            qc = QuantumCircuit(end - start, end - start)
            # Aplica a regra quântica baseada no estado clássico
            qc = apply_quantum_rule_based_on_classical_state(qc, block_state)
            # Adiciona medições aos qubits
            qc.measure(range(end - start), range(end - start))

            # Executa a simulação no backend AerSimulator
            aer_sim = AerSimulator()
            # Transpila o circuito para o simulador Aer
            transpiled_qc = transpile(qc, aer_sim)
            # Executa a simulação e obtém os resultados
            result = aer_sim.run(transpiled_qc, shots=1024, memory=True).result()
            # Obtém as contagens dos resultados
            counts = result.get_counts()
            # Adiciona os resultados do bloco à lista
            block_results.append(counts)

            # Desenho do circuito com título e subtítulo
            title = f'Circuito Quântico {rule_number} - Bloco {block + 1} - Circuito {circuit_count}'
            subtitle = ('Este circuito quântico simula a regra {0}. '
                        'Os qubits são configurados com base no estado inicial clássico e, em seguida, '
                        'medições são realizadas para obter os resultados. Neste circuito, as operações quânticas '
                        'aplicadas incluem portas X (NOT) e medições. As portas X são aplicadas aos qubits quando o '
                        'estado clássico correspondente é 1. Após as operações, os qubits são medidos para determinar '
                        'o estado final do sistema.').format(rule_number)
            # Cria uma figura para o circuito
            fig, ax = plt.subplots(figsize=(12, 6))
            # Define o título da figura
            ax.set_title(title)
            # Adiciona o subtítulo à figura
            ax.text(0.5, -0.1, subtitle, ha='center', va='center', transform=ax.transAxes, wrap=True)
            # Desenha o circuito quântico
            qc.draw('mpl', ax=ax)
            # Exibe a figura
            plt.show()

            # Visualização da esfera de Bloch
            statevector = result.data(0).get('statevector')
            if statevector:
                plot_bloch_multivector(statevector)
                plt.title(f'Esfera de Bloch - Circuito {circuit_count}')
                plt.show()

            # Incrementa o contador de circuitos
            circuit_count += 1
        # Adiciona os resultados do bloco à lista de resultados
        results.append(block_results)
    return results
    """
    Explicações Detalhadas:

    Objetivo da Função:

    Esta função executa simulações quânticas baseadas nos estados clássicos fornecidos.
    Ela pega um DataFrame que representa o estado clássico de um sistema, divide esse estado em blocos menores e, em seguida, cria e simula circuitos quânticos para cada bloco.

    Passo a Passo:

    Parâmetros:

    df_classical (pd.DataFrame): Este é um DataFrame que contém os estados clássicos.
    Imagine uma tabela onde cada linha representa um momento no tempo e cada coluna representa uma célula do autômato.
    block_size (int): O tamanho de cada bloco de células que vamos transformar em qubits. Por exemplo, se o tamanho do bloco for 3, vamos agrupar as células em trios.
    rule_number (int): O número da regra clássica que estamos simulando.

    Criação do Simulador:

    simulator = AerSimulator(): Cria um simulador quântico que vamos usar para executar nossos circuitos.
    Inicialização dos Resultados:

    results = []: Cria uma lista vazia para armazenar os resultados das simulações.
    num_blocks = (len(df_classical.columns) + block_size - 1) // block_size: Calcula quantos blocos vamos precisar.
    Isso depende do número total de células e do tamanho dos blocos.

    Iteração Sobre as Linhas do DataFrame:

    for index, row in df_classical.iterrows(): Itera sobre cada linha da tabela de estados clássicos.
    Cada linha representa um estado em um momento específico.

    Iteração Sobre os Blocos:

    for block in range(num_blocks): Para cada linha, iteramos sobre os blocos de células.
    start = block * block_size: Calcula o índice inicial do bloco.
    end = min(start + block_size, len(df_classical.columns)): Calcula o índice final do bloco.

    Estado do Bloco e Criação do Circuito Quântico:

    block_state = row.values[start:end]: Pega os valores das células que pertencem ao bloco atual.
    qc = QuantumCircuit(end - start, end - start): Cria um circuito quântico com qubits para cada célula no bloco.
    Aplicação da Regra Quântica:

    qc = apply_quantum_rule_based_on_classical_state(qc, block_state): Aplica uma regra quântica baseada no estado clássico do bloco.
    Adição de Medições:

    qc.measure(range(end - start), range(end - start)): Adiciona operações de medição aos qubits.
    Isso significa que vamos verificar o estado final de cada qubit após as operações quânticas.

    Simulação do Circuito:

    aer_sim = AerSimulator(): Cria o simulador de novo (embora isso seja redundante e poderia ser feito fora do loop).
    transpiled_qc = transpile(qc, aer_sim): Transpila (prepara) o circuito para ser executado no simulador.
    result = aer_sim.run(transpiled_qc, shots=1024, memory=True).result(): Executa a simulação e coleta os resultados.
    "Shots" são o número de vezes que executamos o circuito para obter uma distribuição de resultados.
    counts = result.get_counts(): Obtém a contagem de cada resultado possível.
    Armazenamento dos Resultados:

    block_results.append(counts): Adiciona os resultados do bloco à lista de resultados dos blocos.

    Visualização do Circuito e Esfera de Bloch:

    Desenha o circuito quântico com título e subtítulo explicativos.
    Se o vetor de estado estiver disponível, plota a esfera de Bloch para visualizar o estado quântico dos qubits.

    Incremento do Contador de Circuitos:

    circuit_count += 1: Incrementa o contador para o próximo circuito.

    Armazenamento Final dos Resultados:

    results.append(block_results): Adiciona os resultados de todos os blocos da linha atual à lista de resultados finais.

    Retorno dos Resultados:

    return results: Retorna a lista de resultados das simulações quânticas.
    """

# Função para converter resultados em DataFrame
def quantum_results_to_dataframe(quantum_results):
    """
    Converte resultados de simulação quântica em um DataFrame.

    Parameters:
        quantum_results (list): Resultados da simulação quântica.

    Returns:
        pd.DataFrame: DataFrame contendo os resultados.
    """
    # Cria uma lista vazia para armazenar os dados
    data = []
    # Itera sobre cada passo da simulação
    # "step" é o número do passo (como um momento no tempo)
    # "block_results" são os resultados dos blocos nesse passo
    for step, block_results in enumerate(quantum_results):
        # Itera sobre cada bloco de resultados dentro do passo
        # "block_index" é o número do bloco
        # "result" são os resultados desse bloco específico
        for block_index, result in enumerate(block_results):
            # Itera sobre cada estado e sua contagem no resultado do bloco
            # "state" é o estado do qubit (como uma combinação de 0s e 1s)
            # "count" é o número de vezes que esse estado foi observado
            for state, count in result.items():
                # Adiciona um dicionário com as informações do passo, bloco, estado e contagem à lista de dados
                data.append({"Step": step, "Block": block_index, "State": state, "Count": count})
    # Converte a lista de dados em um DataFrame e retorna
    return pd.DataFrame(data)
    """
    Explicações Detalhadas:

    Objetivo da Função:

    Esta função pega os resultados de uma simulação quântica (que são armazenados em listas e dicionários)
    e organiza esses resultados em um formato de tabela chamado DataFrame, que é mais fácil de analisar e visualizar.

    Passo a Passo:

    Parâmetros:

    quantum_results (list): Esta é uma lista que contém os resultados da simulação quântica.
    Cada item na lista representa um passo no tempo e contém os resultados de vários blocos.

    Criação da Lista de Dados:

    data = []: Cria uma lista vazia chamada data. Vamos usar essa lista para armazenar nossos resultados de forma organizada.

    Iteração Sobre os Passos da Simulação:

    for step, block_results in enumerate(quantum_results): Itera sobre cada passo na simulação. A função enumerate nos dá o índice (step)
    e o valor (block_results) de cada item na lista quantum_results.
    step é como o número do passo (imagine um momento específico no tempo).
    block_results são os resultados de todos os blocos naquele passo específico.

    Iteração Sobre os Blocos de Resultados:

    for block_index, result in enumerate(block_results): Itera sobre cada bloco de resultados dentro de um passo.
    Novamente, usamos enumerate para obter o índice (block_index) e o valor (result) de cada item em block_results.
    block_index é como o número do bloco dentro daquele passo.
    result é um dicionário que contém os estados e suas contagens para aquele bloco.

    Iteração Sobre os Estados e Contagens:

    for state, count in result.items(): Itera sobre cada estado e sua contagem no resultado do bloco.
    state é o estado do qubit, que é uma combinação de 0s e 1s (como "001", "110", etc.).
    count é o número de vezes que esse estado foi observado na simulação.

    Armazenamento dos Dados:

    data.append({"Step": step, "Block": block_index, "State": state, "Count": count}): Adiciona um dicionário à lista data
    com as informações do passo, bloco, estado e contagem.

    Conversão para DataFrame:

    return pd.DataFrame(data): Converte a lista data em um DataFrame usando a biblioteca pandas e retorna esse DataFrame.
    """

def join_quantum_blocks(df_quantum, block_size, total_size):
    """
    Junta blocos de resultados quânticos em um DataFrame total.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame com resultados quânticos.
        block_size (int): Tamanho do bloco de células.
        total_size (int): Tamanho total do autômato.

    Returns:
        pd.DataFrame: DataFrame combinado dos blocos.
    """
    # Obtém o número máximo de passos na simulação e adiciona 1
    # Aqui, estamos descobrindo quantos momentos (passos) diferentes existem na simulação.
    steps = df_quantum['Step'].max() + 1

    # Cria uma lista vazia para armazenar os dados combinados
    combined_data = []

    # Itera sobre cada passo da simulação
    # "step" é o número do passo (como um momento específico no tempo)
    for step in range(steps):
        # Obtém os dados do DataFrame que correspondem ao passo atual
        step_data = df_quantum[df_quantum['Step'] == step]

        # Cria uma lista vazia para armazenar os dados combinados do passo atual
        combined_row = []

        # Itera sobre cada bloco de resultados dentro do passo atual
        # "block" é o número do bloco
        for block in range(step_data['Block'].max() + 1):
            # Obtém os dados do DataFrame que correspondem ao bloco atual dentro do passo atual
            block_data = step_data[step_data['Block'] == block]

            # Verifica se o bloco não está vazio
            if not block_data.empty:
                # Conta quantas vezes cada estado aparece no bloco
                state_counts = block_data.groupby('State')['Count'].sum()

                # Encontra o estado que aparece com mais frequência no bloco
                most_frequent_state = state_counts.idxmax()

                # Converte o estado mais frequente (uma string de 0s e 1s) em uma lista de inteiros
                combined_row.extend([int(bit) for bit in most_frequent_state])
            else:
                # Se o bloco estiver vazio, adiciona zeros à lista combinada
                combined_row.extend([0] * block_size)

        # Adiciona o número do passo e os dados combinados à lista total
        combined_data.append([step] + combined_row[:total_size])

    # Cria uma lista de nomes de colunas, começando com 'Step' e seguido por 'Qubit_0', 'Qubit_1', etc.
    columns = ['Step'] + [f'Qubit_{i}' for i in range(total_size)]

    # Converte a lista de dados combinados em um DataFrame com as colunas definidas e retorna
    return pd.DataFrame(combined_data, columns=columns)
"""
Explicações Detalhadas:

Objetivo da Função:

Esta função pega os resultados da simulação quântica (que estão divididos em blocos) e os junta em um único DataFrame,
organizado por passos (momentos no tempo) e estados dos qubits.

Passo a Passo:

Parâmetros:

df_quantum (pd.DataFrame): Este é o DataFrame que contém os resultados da simulação quântica.
block_size (int): Este é o número de células (ou qubits) em cada bloco.
total_size (int): Este é o tamanho total do autômato, ou seja, o número total de qubits.

Número de Passos na Simulação:

steps = df_quantum['Step'].max() + 1: Descobre o número máximo de passos na simulação e adiciona 1 para contar desde zero.
Isso nos diz quantos momentos diferentes existem na simulação.

Criação da Lista de Dados Combinados:

combined_data = []: Cria uma lista vazia chamada combined_data para armazenar os dados combinados de todos os blocos.

Iteração Sobre os Passos da Simulação:

for step in range(steps): Itera sobre cada passo na simulação.
step_data = df_quantum[df_quantum['Step'] == step]: Filtra o DataFrame df_quantum para obter apenas os dados que correspondem ao passo atual.

Criação da Linha Combinada:

combined_row = []: Cria uma lista vazia chamada combined_row para armazenar os dados combinados do passo atual.

Iteração Sobre os Blocos de Resultados:

for block in range(step_data['Block'].max() + 1): Itera sobre cada bloco de resultados dentro do passo atual.
block_data = step_data[step_data['Block'] == block]: Filtra os dados do passo atual para obter apenas os dados que correspondem ao bloco atual.

Verificação e Processamento dos Blocos:

if not block_data.empty: Verifica se o bloco não está vazio.
state_counts = block_data.groupby('State')['Count'].sum(): Conta quantas vezes cada estado aparece no bloco.
most_frequent_state = state_counts.idxmax(): Encontra o estado que aparece com mais frequência no bloco.
combined_row.extend([int(bit) for bit in most_frequent_state]): Converte o estado mais frequente (uma string de 0s e 1s) em uma lista de inteiros e adiciona à lista combinada.
else: combined_row.extend([0] * block_size): Se o bloco estiver vazio, adiciona zeros à lista combinada.

Adição dos Dados Combinados à Lista Total:

combined_data.append([step] + combined_row[:total_size]): Adiciona o número do passo e os dados combinados (limitados ao tamanho total) à lista combined_data.
Criação das Colunas e Retorno do DataFrame:

columns = ['Step'] + [f'Qubit_{i}' for i in range(total_size)]: Cria uma lista de nomes de colunas, começando com 'Step' e seguido por 'Qubit_0', 'Qubit_1', etc.
return pd.DataFrame(combined_data, columns=columns): Converte a lista combined_data em um DataFrame com as colunas definidas e retorna esse DataFrame.
"""

# Simulações clássicas

# Inicia a medição do tempo de execução para a simulação da regra 30
start_time = time.time()
# Executa a simulação clássica da regra 30
history_30_classical = simulate_classical(30)
# Calcula o tempo de execução da simulação da regra 30
classical_30_time = time.time() - start_time

# Inicia a medição do tempo de execução para a simulação da regra 60
start_time = time.time()
# Executa a simulação clássica da regra 60
history_60_classical = simulate_classical(60)
# Calcula o tempo de execução da simulação da regra 60
classical_60_time = time.time() - start_time

# Inicia a medição do tempo de execução para a simulação da regra 90
start_time = time.time()
# Executa a simulação clássica da regra 90
history_90_classical = simulate_classical(90)
# Calcula o tempo de execução da simulação da regra 90
classical_90_time = time.time() - start_time

# Armazenar os resultados em DataFrames

# Converte os resultados da simulação clássica da regra 30 em um DataFrame (tabela)
df_30_classical = pd.DataFrame(history_30_classical)
# Converte os resultados da simulação clássica da regra 60 em um DataFrame (tabela)
df_60_classical = pd.DataFrame(history_60_classical)
# Converte os resultados da simulação clássica da regra 90 em um DataFrame (tabela)
df_90_classical = pd.DataFrame(history_90_classical)

# Executar simulações quânticas baseadas nos estados clássicos

# Define o tamanho do bloco para a simulação quântica
block_size = 7
# Define o tamanho total com base no número de colunas do DataFrame clássico da regra 30
total_size = len(df_30_classical.columns)

# Inicia a medição do tempo de execução para a simulação quântica da regra 30
start_time = time.time()
# Executa a simulação quântica da regra 30
counts_30_quantum = run_quantum_simulation_from_classical(df_30_classical, block_size, 30)
# Calcula o tempo de execução da simulação quântica da regra 30
quantum_30_time = time.time() - start_time

# Inicia a medição do tempo de execução para a simulação quântica da regra 60
start_time = time.time()
# Executa a simulação quântica da regra 60
counts_60_quantum = run_quantum_simulation_from_classical(df_60_classical, block_size, 60)
# Calcula o tempo de execução da simulação quântica da regra 60
quantum_60_time = time.time() - start_time

# Inicia a medição do tempo de execução para a simulação quântica da regra 90
start_time = time.time()
# Executa a simulação quântica da regra 90
counts_90_quantum = run_quantum_simulation_from_classical(df_90_classical, block_size, 90)
# Calcula o tempo de execução da simulação quântica da regra 90
quantum_90_time = time.time() - start_time

# Converter os resultados das simulações quânticas em DataFrames

# Converte os resultados da simulação quântica da regra 30 em um DataFrame (tabela)
df_30_quantum = quantum_results_to_dataframe(counts_30_quantum)
# Converte os resultados da simulação quântica da regra 60 em um DataFrame (tabela)
df_60_quantum = quantum_results_to_dataframe(counts_60_quantum)
# Converte os resultados da simulação quântica da regra 90 em um DataFrame (tabela)
df_90_quantum = quantum_results_to_dataframe(counts_90_quantum)

# Juntar blocos quânticos e criar DataFrames totais

# Junta os blocos da simulação quântica da regra 30 em um DataFrame (tabela) total
df_30_quantum_combined = join_quantum_blocks(df_30_quantum, block_size, total_size)
# Junta os blocos da simulação quântica da regra 60 em um DataFrame (tabela) total
df_60_quantum_combined = join_quantum_blocks(df_60_quantum, block_size, total_size)
# Junta os blocos da simulação quântica da regra 90 em um DataFrame (tabela) total
df_90_quantum_combined = join_quantum_blocks(df_90_quantum, block_size, total_size)

# Salvar os DataFrames em arquivos CSV

# Salva o DataFrame da simulação clássica da regra 30 em um arquivo CSV
df_30_classical.to_csv('/content/df_30_classical.csv', index=False)
# Salva o DataFrame da simulação clássica da regra 60 em um arquivo CSV
df_60_classical.to_csv('/content/df_60_classical.csv', index=False)
# Salva o DataFrame da simulação clássica da regra 90 em um arquivo CSV
df_90_classical.to_csv('/content/df_90_classical.csv', index=False)
# Salva o DataFrame combinado da simulação quântica da regra 30 em um arquivo CSV
df_30_quantum_combined.to_csv('/content/df_30_quantum_combined.csv', index=False)
# Salva o DataFrame combinado da simulação quântica da regra 60 em um arquivo CSV
df_60_quantum_combined.to_csv('/content/df_60_quantum_combined.csv', index=False)
# Salva o DataFrame combinado da simulação quântica da regra 90 em um arquivo CSV
df_90_quantum_combined.to_csv('/content/df_90_quantum_combined.csv', index=False)

"""
Explicações Detalhadas:

Simulações Clássicas:

Medição do Tempo de Execução:

start_time = time.time(): Inicia a contagem do tempo antes de executar a simulação.

Execução das Simulações:

history_30_classical = simulate_classical(30): Executa a simulação clássica para a regra 30.
history_60_classical = simulate_classical(60): Executa a simulação clássica para a regra 60.
history_90_classical = simulate_classical(90): Executa a simulação clássica para a regra 90.

Cálculo do Tempo de Execução:

classical_30_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 30.
classical_60_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 60.
classical_90_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 90.

Armazenar os Resultados em DataFrames:

df_30_classical = pd.DataFrame(history_30_classical): Converte os resultados da simulação clássica da regra 30 em um DataFrame (tabela).
df_60_classical = pd.DataFrame(history_60_classical): Converte os resultados da simulação clássica da regra 60 em um DataFrame (tabela).
df_90_classical = pd.DataFrame(history_90_classical): Converte os resultados da simulação clássica da regra 90 em um DataFrame (tabela).

Executar Simulações Quânticas Baseadas nos Estados Clássicos:

Definir Tamanho do Bloco e Tamanho Total:

block_size = 7: Define que cada bloco terá 7 células (ou qubits).
total_size = len(df_30_classical.columns): Define o tamanho total com base no número de colunas do DataFrame clássico da regra 30.

Medição do Tempo de Execução:

start_time = time.time(): Inicia a contagem do tempo antes de executar a simulação.

Execução das Simulações:

counts_30_quantum = run_quantum_simulation_from_classical(df_30_classical, block_size, 30): Executa a simulação quântica para a regra 30.
counts_60_quantum = run_quantum_simulation_from_classical(df_60_classical, block_size, 60): Executa a simulação quântica para a regra 60.
counts_90_quantum = run_quantum_simulation_from_classical(df_90_classical, block_size, 90): Executa a simulação quântica para a regra 90.

Cálculo do Tempo de Execução:

quantum_30_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 30.
quantum_60_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 60.
quantum_90_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 90.

Converter os Resultados das Simulações Quânticas em DataFrames:

df_30_quantum = quantum_results_to_dataframe(counts_30_quantum): Converte os resultados da simulação quântica da regra 30 em um DataFrame (tabela).
df_60_quantum = quantum_results_to_dataframe(counts_60_quantum): Converte os resultados da simulação quântica da regra 60 em um DataFrame (tabela).
df_90_quantum = quantum_results_to_dataframe(counts_90_quantum): Converte os resultados da simulação quântica da regra 90 em um DataFrame (tabela).

Juntar Blocos Quânticos e Criar DataFrames Totais:

df_30_quantum_combined = join_quantum_blocks(df_30_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 30 em um DataFrame total.
df_60_quantum_combined = join_quantum_blocks(df_60_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 60 em um DataFrame total.
df_90_quantum_combined = join_quantum_blocks(df_90_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 90 em um DataFrame total.

Salvar os DataFrames em Arquivos CSV:

df_30_classical.to_csv('/content/df_30_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 30 em um arquivo CSV.
df_60_classical.to_csv('/content/df_60_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 60 em um arquivo CSV.
df_90_classical.to_csv('/content/df_90_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 90 em um arquivo CSV.
df_30_quantum_combined.to_csv('/content/df_30_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 30 em um arquivo CSV.
df_60_quantum_combined.to_csv('/content/df_60_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 60 em um arquivo CSV.
df_90_quantum_combined.to_csv('/content/df_90_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 90 em um arquivo CSV.
"""

# Função para plotar simulações clássicas
def plot_classical_simulation(history, title):
    """
    Plota a simulação clássica usando Seaborn.

    Parameters:
        history (np.array): Histórico dos estados do autômato.
        title (str): Título do gráfico.
    """
    plt.figure(figsize=(10, 10))  # Cria uma figura para o gráfico
    sns.heatmap(history, cmap='binary', cbar=True)  # Cria um mapa de calor para a simulação clássica
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Posição')  # Define o rótulo do eixo x
    plt.ylabel('Passo')  # Define o rótulo do eixo y
    plt.show()  # Exibe o gráfico

# Função para plotar simulações quânticas
def plot_quantum_simulation(df, title):
    """
    Plota a simulação quântica usando Seaborn.

    Parameters:
        df (pd.DataFrame): DataFrame contendo os resultados da simulação quântica.
        title (str): Título do gráfico.
    """
    steps = df['Step'].unique()  # Obtém os passos únicos da simulação
    df_heatmap = df.drop(columns='Step').set_index(steps)  # Prepara os dados para o mapa de calor
    plt.figure(figsize=(10, 10))  # Cria uma figura para o gráfico
    sns.heatmap(df_heatmap, cmap='viridis', cbar=True)  # Cria um mapa de calor para a simulação quântica
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Qubit')  # Define o rótulo do eixo x
    plt.ylabel('Passo')  # Define o rótulo do eixo y
    plt.show()  # Exibe o gráfico

# Plotagens das regras clássicas
plot_classical_simulation(df_30_classical.values, 'Simulação Clássica da Regra 30')  # Plota a simulação clássica da regra 30
plot_classical_simulation(df_60_classical.values, 'Simulação Clássica da Regra 60')  # Plota a simulação clássica da regra 60
plot_classical_simulation(df_90_classical.values, 'Simulação Clássica da Regra 90')  # Plota a simulação clássica da regra 90

# Plotagens das regras quânticas
plot_quantum_simulation(df_30_quantum_combined, 'Simulação Quântica da Regra 30')  # Plota a simulação quântica da regra 30
plot_quantum_simulation(df_60_quantum_combined, 'Simulação Quântica da Regra 60')  # Plota a simulação quântica da regra 60
plot_quantum_simulation(df_90_quantum_combined, 'Simulação Quântica da Regra 90')  # Plota a simulação quântica da regra 90

# Função para calcular a entropia dos resultados quânticos
def calculate_entropy(df_quantum):
    """
    Calcula a entropia dos resultados quânticos.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame contendo os resultados quânticos.

    Returns:
        float: Entropia média.
    """
    entropies = []  # Lista para armazenar as entropias
    # Itera sobre cada passo da simulação
    for step in df_quantum['Step'].unique():
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        counts = step_data.drop(columns=['Step']).values.flatten()  # Obtém as contagens dos estados
        entropies.append(entropy(counts[counts > 0]))  # Calcula a entropia e adiciona à lista
    return np.mean(entropies)  # Retorna a entropia média

# Calcula a entropia para cada regra quântica
entropy_30_quantum = calculate_entropy(df_30_quantum_combined)  # Calcula a entropia da simulação quântica da regra 30
entropy_60_quantum = calculate_entropy(df_60_quantum_combined)  # Calcula a entropia da simulação quântica da regra 60
entropy_90_quantum = calculate_entropy(df_90_quantum_combined)  # Calcula a entropia da simulação quântica da regra 90

# Imprime as entropias calculadas
print(f"Entropia da Regra Quântica 30: {entropy_30_quantum}")  # Imprime a entropia da regra 30
print(f"Entropia da Regra Quântica 60: {entropy_60_quantum}")  # Imprime a entropia da regra 60
print(f"Entropia da Regra Quântica 90: {entropy_90_quantum}")  # Imprime a entropia da regra 90

# Robustez ao Ruído
# Adiciona ruído aos resultados quânticos e recalcula a entropia
noisy_counts_30_quantum = add_noise_to_quantum_results(counts_30_quantum)  # Adiciona ruído à simulação quântica da regra 30
noisy_counts_60_quantum = add_noise_to_quantum_results(counts_60_quantum)  # Adiciona ruído à simulação quântica da regra 60
noisy_counts_90_quantum = add_noise_to_quantum_results(counts_90_quantum)  # Adiciona ruído à simulação quântica da regra 90

df_noisy_30_quantum = quantum_results_to_dataframe(noisy_counts_30_quantum)  # Converte os resultados ruidosos da regra 30 em um DataFrame
df_noisy_60_quantum = quantum_results_to_dataframe(noisy_counts_60_quantum)  # Converte os resultados ruidosos da regra 60 em um DataFrame
df_noisy_90_quantum = quantum_results_to_dataframe(noisy_counts_90_quantum)  # Converte os resultados ruidosos da regra 90 em um DataFrame

df_noisy_30_quantum_combined = join_quantum_blocks(df_noisy_30_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 30
df_noisy_60_quantum_combined = join_quantum_blocks(df_noisy_60_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 60
df_noisy_90_quantum_combined = join_quantum_blocks(df_noisy_90_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 90

noisy_entropy_30_quantum = calculate_entropy(df_noisy_30_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 30
noisy_entropy_60_quantum = calculate_entropy(df_noisy_60_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 60
noisy_entropy_90_quantum = calculate_entropy(df_noisy_90_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 90

# Imprime as entropias calculadas com ruído
print(f"Entropia da Regra Quântica 30 com Ruído: {noisy_entropy_30_quantum}")  # Imprime a entropia ruidosa da regra 30
print(f"Entropia da Regra Quântica 60 com Ruído: {noisy_entropy_60_quantum}")  # Imprime a entropia ruidosa da regra 60
print(f"Entropia da Regra Quântica 90 com Ruído: {noisy_entropy_90_quantum}")  # Imprime a entropia ruidosa da regra 90

"""
Explicações Detalhadas:

Simulações Clássicas:

Medição do Tempo de Execução:

start_time = time.time(): Inicia a contagem do tempo antes de executar a simulação.

Execução das Simulações:

history_30_classical = simulate_classical(30): Executa a simulação clássica para a regra 30.
history_60_classical = simulate_classical(60): Executa a simulação clássica para a regra 60.
history_90_classical = simulate_classical(90): Executa a simulação clássica para a regra 90.

Cálculo do Tempo de Execução:

classical_30_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 30.
classical_60_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 60.
classical_90_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação da regra 90.

Armazenar os Resultados em DataFrames:

df_30_classical = pd.DataFrame(history_30_classical): Converte os resultados da simulação clássica da regra 30 em um DataFrame (tabela).
df_60_classical = pd.DataFrame(history_60_classical): Converte os resultados da simulação clássica da regra 60 em um DataFrame (tabela).
df_90_classical = pd.DataFrame(history_90_classical): Converte os resultados da simulação clássica da regra 90 em um DataFrame (tabela).

Executar Simulações Quânticas Baseadas nos Estados Clássicos:

Definir Tamanho do Bloco e Tamanho Total:

block_size = 7: Define que cada bloco terá 7 células (ou qubits).
total_size = len(df_30_classical.columns): Define o tamanho total com base no número de colunas do DataFrame clássico da regra 30.

Medição do Tempo de Execução:

start_time = time.time(): Inicia a contagem do tempo antes de executar a simulação.

Execução das Simulações:

counts_30_quantum = run_quantum_simulation_from_classical(df_30_classical, block_size, 30): Executa a simulação quântica para a regra 30.
counts_60_quantum = run_quantum_simulation_from_classical(df_60_classical, block_size, 60): Executa a simulação quântica para a regra 60.
counts_90_quantum = run_quantum_simulation_from_classical(df_90_classical, block_size, 90): Executa a simulação quântica para a regra 90.

Cálculo do Tempo de Execução:

quantum_30_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 30.
quantum_60_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 60.
quantum_90_time = time.time() - start_time: Calcula o tempo que levou para executar a simulação quântica da regra 90.

Converter os Resultados das Simulações Quânticas em DataFrames:

df_30_quantum = quantum_results_to_dataframe(counts_30_quantum): Converte os resultados da simulação quântica da regra 30 em um DataFrame (tabela).
df_60_quantum = quantum_results_to_dataframe(counts_60_quantum): Converte os resultados da simulação quântica da regra 60 em um DataFrame (tabela).
df_90_quantum = quantum_results_to_dataframe(counts_90_quantum): Converte os resultados da simulação quântica da regra 90 em um DataFrame (tabela).

Juntar Blocos Quânticos e Criar DataFrames Totais:

df_30_quantum_combined = join_quantum_blocks(df_30_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 30 em um DataFrame total.
df_60_quantum_combined = join_quantum_blocks(df_60_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 60 em um DataFrame total.
df_90_quantum_combined = join_quantum_blocks(df_90_quantum, block_size, total_size): Junta os blocos da simulação quântica da regra 90 em um DataFrame total.

Salvar os DataFrames em Arquivos CSV:

df_30_classical.to_csv('/content/df_30_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 30 em um arquivo CSV.
df_60_classical.to_csv('/content/df_60_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 60 em um arquivo CSV.
df_90_classical.to_csv('/content/df_90_classical.csv', index=False): Salva o DataFrame da simulação clássica da regra 90 em um arquivo CSV.
df_30_quantum_combined.to_csv('/content/df_30_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 30 em um arquivo CSV.
df_60_quantum_combined.to_csv('/content/df_60_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 60 em um arquivo CSV.
df_90_quantum_combined.to_csv('/content/df_90_quantum_combined.csv', index=False): Salva o DataFrame combinado da simulação quântica da regra 90 em um arquivo CSV.
"""
# Função para plotar entropia por passo
def plot_entropy_per_step(df_quantum, title):
    """
    Plota a entropia para cada passo da simulação quântica.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame contendo os resultados quânticos.
        title (str): Título do gráfico.
    """
    entropies = []  # Lista para armazenar as entropias
    steps = df_quantum['Step'].unique()  # Obtém os passos únicos da simulação

    # Itera sobre cada passo da simulação
    for step in steps:
        # Obtém os dados do passo atual
        step_data = df_quantum[df_quantum['Step'] == step]
        # Obtém as contagens dos estados, ignorando a coluna 'Step'
        counts = step_data.drop(columns=['Step']).values.flatten()
        # Calcula a entropia dos estados no passo atual e adiciona à lista de entropias
        entropies.append(entropy(counts[counts > 0]))

    # Plota a entropia para cada passo da simulação
    plt.figure(figsize=(10, 6))  # Define o tamanho da figura do gráfico
    plt.plot(steps, entropies, marker='o')  # Cria um gráfico de linha com os passos e entropias
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Passo')  # Define o rótulo do eixo x
    plt.ylabel('Entropia')  # Define o rótulo do eixo y
    plt.grid(True)  # Adiciona uma grade ao gráfico para melhor visualização
    plt.show()  # Exibe o gráfico

    # Salva os dados de entropia em um DataFrame e exporta para CSV
    df_entropy = pd.DataFrame({'Step': steps, 'Entropy': entropies})  # Cria um DataFrame com os passos e entropias
    df_entropy.to_csv(f'/content/{title}.csv', index=False)  # Salva o DataFrame em um arquivo CSV
"""
Explicações Detalhadas:

Definição da Função:

def plot_entropy_per_step(df_quantum, title):: Define a função plot_entropy_per_step que plota a entropia para cada passo da simulação quântica.

Docstring:

'Plota a entropia para cada passo da simulação quântica. Parameters: df_quantum (pd.DataFrame): DataFrame contendo os resultados quânticos. title (str): Título do gráfico.' : Explica o que a função faz e quais são seus parâmetros.
Inicialização de Variáveis:

entropies = []: Cria uma lista vazia para armazenar as entropias calculadas.
steps = df_quantum['Step'].unique(): Obtém uma lista de todos os passos únicos da simulação quântica.
Cálculo da Entropia por Passo:

for step in steps:: Itera sobre cada passo da simulação.
step_data = df_quantum[df_quantum['Step'] == step]: Obtém os dados do DataFrame para o passo atual.
counts = step_data.drop(columns=['Step']).values.flatten(): Remove a coluna 'Step' e obtém os valores das contagens, convertendo-os em um array unidimensional.
entropies.append(entropy(counts[counts > 0])): Calcula a entropia para os estados do passo atual (ignorando zeros) e adiciona à lista de entropias.
Plotagem do Gráfico:

plt.figure(figsize=(10, 6)): Define o tamanho da figura do gráfico (10 polegadas por 6 polegadas).
plt.plot(steps, entropies, marker='o'): Plota os passos no eixo x e as entropias no eixo y, usando círculos como marcadores.
plt.title(title): Define o título do gráfico.
plt.xlabel('Passo'): Define o rótulo do eixo x como 'Passo'.
plt.ylabel('Entropia'): Define o rótulo do eixo y como 'Entropia'.
plt.grid(True): Adiciona uma grade ao gráfico para facilitar a visualização dos dados.
plt.show(): Exibe o gráfico.

Salvar Dados em CSV:

df_entropy = pd.DataFrame({'Step': steps, 'Entropy': entropies}): Cria um DataFrame com duas colunas: 'Step' para os passos e 'Entropy' para as entropias calculadas.
df_entropy.to_csv(f'/content/{title}.csv', index=False): Salva o DataFrame em um arquivo CSV, usando o título do gráfico como parte do nome do arquivo.
"""

# Plota a entropia por passo e salva os dados em tabelas CSV

# Chama a função para plotar a entropia da simulação quântica da regra 30
# 'df_30_quantum_combined' contém os resultados combinados da simulação quântica para a regra 30
# 'Entropia_por_Passo_Regra_Quantica_30' é o título do gráfico e o nome do arquivo CSV
plot_entropy_per_step(df_30_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_30')

# Chama a função para plotar a entropia da simulação quântica da regra 60
# 'df_60_quantum_combined' contém os resultados combinados da simulação quântica para a regra 60
# 'Entropia_por_Passo_Regra_Quantica_60' é o título do gráfico e o nome do arquivo CSV
plot_entropy_per_step(df_60_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_60')

# Chama a função para plotar a entropia da simulação quântica da regra 90
# 'df_90_quantum_combined' contém os resultados combinados da simulação quântica para a regra 90
# 'Entropia_por_Passo_Regra_Quantica_90' é o título do gráfico e o nome do arquivo CSV
plot_entropy_per_step(df_90_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_90')
"""
Explicações Detalhadas:

Definição do Título e Dados:

plot_entropy_per_step(df_30_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_30'): Esta linha chama a função plot_entropy_per_step para a regra 30. Aqui, df_30_quantum_combined é o DataFrame que contém os resultados da simulação quântica combinada para a regra 30. O título 'Entropia_por_Passo_Regra_Quantica_30' será usado tanto para o gráfico quanto para o nome do arquivo CSV onde os dados serão salvos.
Chamada da Função para a Regra 60:

plot_entropy_per_step(df_60_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_60'): Semelhante à linha anterior, esta chama a função plot_entropy_per_step para a regra 60, usando df_60_quantum_combined como o DataFrame de resultados e 'Entropia_por_Passo_Regra_Quantica_60' como o título do gráfico e nome do arquivo CSV.
Chamada da Função para a Regra 90:

plot_entropy_per_step(df_90_quantum_combined, 'Entropia_por_Passo_Regra_Quantica_90'): Esta linha chama a função plot_entropy_per_step para a regra 90, usando df_90_quantum_combined como o DataFrame de resultados e 'Entropia_por_Passo_Regra_Quantica_90' como o título do gráfico e nome do arquivo CSV.

Detalhamento:

Função plot_entropy_per_step:

Essa função calcula a entropia para cada passo da simulação quântica.
Cria um gráfico mostrando como a entropia muda a cada passo.
Salva os dados de entropia em um arquivo CSV para que possamos revisar ou usar esses dados mais tarde.
Entropia:

A entropia é uma medida que diz quão "bagunçados" ou "desorganizados" os estados dos qubits estão. É uma maneira de entender a quantidade de incerteza ou informação em um sistema quântico.

DataFrame df_30_quantum_combined, df_60_quantum_combined, df_90_quantum_combined:

Esses DataFrames contêm os resultados das simulações quânticas após combinarmos os blocos de células. Eles mostram como os estados dos qubits mudam ao longo do tempo, passo a passo, para as regras 30, 60 e 90.

Gráficos e CSV:

O gráfico nos ajuda a visualizar a entropia a cada passo da simulação, facilitando a compreensão de como a incerteza muda ao longo do tempo.
O arquivo CSV salva os mesmos dados do gráfico, mas em um formato que podemos abrir em programas como Excel para analisar mais detalhadamente ou compartilhar com outras pessoas.
"""
# Função para salvar os tempos de execução em um DataFrame e exportar para CSV
def save_execution_times(classical_times, quantum_times, filename):
    """
    Salva os tempos de execução das simulações clássicas e quânticas em um DataFrame e exporta para CSV.

    Parameters:
        classical_times (dict): Dicionário contendo os tempos de execução das simulações clássicas.
        quantum_times (dict): Dicionário contendo os tempos de execução das simulações quânticas.
        filename (str): Nome do arquivo CSV para salvar os dados.
    """
    # Cria um DataFrame com os tempos de execução
    # Aqui estamos criando uma tabela com três colunas: 'Regra', 'Tempo Clássico (s)', e 'Tempo Quântico (s)'
    df_times = pd.DataFrame({
        'Regra': list(classical_times.keys()),  # Pega as regras das simulações clássicas (ex: 30, 60, 90)
        'Tempo Clássico (s)': list(classical_times.values()),  # Pega os tempos de execução das simulações clássicas
        'Tempo Quântico (s)': list(quantum_times.values())  # Pega os tempos de execução das simulações quânticas
    })
    # Exporta o DataFrame para um arquivo CSV
    # Salva a tabela em um arquivo com o nome dado (ex: 'tempos_execucao.csv')
    df_times.to_csv(filename, index=False)  # index=False significa que não queremos salvar os números das linhas
    """
    Explicações Detalhadas:

    Definição da Função:

    def save_execution_times(classical_times, quantum_times, filename):: Define uma função chamada save_execution_times que aceita três parâmetros: classical_times, quantum_times e filename.
    Comentário da Função:

    Criação do DataFrame:

    df_times = pd.DataFrame({ ... }): Cria uma tabela (DataFrame) usando a biblioteca pandas. Essa tabela tem três colunas: 'Regra', 'Tempo Clássico (s)', e 'Tempo Quântico (s)'.
    'Regra': list(classical_times.keys()): Pega as chaves (os números das regras) do dicionário classical_times e cria uma lista com elas. Essas chaves são usadas para a coluna 'Regra'.
    'Tempo Clássico (s)': list(classical_times.values()): Pega os valores (os tempos de execução) do dicionário classical_times e cria uma lista com eles. Esses valores são usados para a coluna 'Tempo Clássico (s)'.
    'Tempo Quântico (s)': list(quantum_times.values()): Pega os valores (os tempos de execução) do dicionário quantum_times e cria uma lista com eles. Esses valores são usados para a coluna 'Tempo Quântico (s)'.

    Exportação para CSV:

    df_times.to_csv(filename, index=False): Salva a tabela em um arquivo CSV com o nome dado pelo parâmetro filename. O argumento index=False significa que não queremos salvar os números das linhas (índices) no arquivo CSV.

    Detalhamento:

    Dicionário classical_times e quantum_times:

    Estes dicionários contêm os tempos de execução das simulações clássicas e quânticas, respectivamente. Cada chave no dicionário é o número da regra (ex: 30, 60, 90) e cada valor é o tempo de execução em segundos.

    DataFrame df_times:

    Um DataFrame é como uma tabela com linhas e colunas. Aqui, estamos criando uma tabela onde cada linha representa uma regra de simulação, e as colunas mostram o tempo que cada simulação levou para rodar, tanto a clássica quanto a quântica.

    Arquivo CSV:

    Um arquivo CSV (Comma-Separated Values) é um tipo de arquivo que armazena dados em formato de tabela, onde cada linha é um registro e cada coluna é um campo, separados por vírgulas. Esses arquivos são fáceis de abrir e editar em programas como Excel.
    """

# Salva os tempos de execução em tabelas CSV

# Cria um dicionário para armazenar os tempos de execução das simulações clássicas
classical_times = {
    '30': classical_30_time,  # Tempo de execução da simulação clássica da regra 30
    '60': classical_60_time,  # Tempo de execução da simulação clássica da regra 60
    '90': classical_90_time   # Tempo de execução da simulação clássica da regra 90
}

# Cria um dicionário para armazenar os tempos de execução das simulações quânticas
quantum_times = {
    '30': quantum_30_time,  # Tempo de execução da simulação quântica da regra 30
    '60': quantum_60_time,  # Tempo de execução da simulação quântica da regra 60
    '90': quantum_90_time   # Tempo de execução da simulação quântica da regra 90
}

# Chama a função save_execution_times para salvar os tempos de execução em um arquivo CSV
save_execution_times(classical_times, quantum_times, '/content/tempos_de_execucao.csv')

"""
Explicações Detalhadas:

Comentário do Bloco de Código:

# Salva os tempos de execução em tabelas CSV: Explica que o bloco de código abaixo vai salvar os tempos de execução das simulações clássicas e quânticas em um arquivo CSV.


"""




